This is a quick example of the basic usage of the refraction component in the Agent Lifecycle Toolkit, accompanying the video demonstration here: https://ibm.box.com/v/refraction-demo.

We have a set of hr tools below, with the usual api specs containing their input and output schemas. The tools specs could be describing python tools by themselves or direct call to APIs or python functions wrapping APIs.

In [10]:
tool_specs = [
    {
        "name": "w3",
        "description": "Get employee information from w3",
        "parameters": {
            "type": "object",
            "properties": {
                "email": {
                    "type": "string",
                    "description": "Employee email",
                }
            },
            "required": ["email"],
        },
        "output_parameters": {
            "type": "object",
            "properties": {
                "id": {
                    "type": "id",
                    "description": "Employee id",
                }
            },
        },
    },
    {
        "name": "author_workbench",
        "description": "Get employee publication information",
        "parameters": {
            "type": "object",
            "properties": {
                "id": {
                    "type": "string",
                    "description": "Employee id",
                }
            },
            "required": ["id"],
        },
        "output_parameters": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "papers": {
                        "type": "object",
                        "description": "Paper information",
                    }
                },
            },
        },
    },
    {
        "name": "hr_bot",
        "description": "Get employee information from HR",
        "parameters": {
            "type": "object",
            "properties": {
                "id": {
                    "type": "string",
                    "description": "Employee id",
                },
                "email": {
                    "type": "string",
                    "description": "Employee email",
                },
            },
        },
        "output_parameters": {
            "type": "object",
            "properties": {
                "id": {
                    "type": "string",
                    "description": "Employee id",
                },
                "info": {
                    "type": "object",
                    "description": "Employee information",
                },
            },
        },
    },
    {
        "name": "concur",
        "description": "Apply for travel approval",
        "parameters": {
            "type": "object",
            "properties": {
                "employee_info": {
                    "type": "object",
                    "description": "Employee information",
                },
                "travel_justification": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "paper": {
                                "type": "object",
                                "description": "Paper information",
                            }
                        },
                    },
                },
            },
            "required": ["employee_info", "travel_justification"],
        },
    },
]

Let's say your agent has produced here a series of calls to put in a travel approval application for a conference you are trying to go to. Are these tool calls proper? Are there calls here that are not necessary? Is there a better way to use the given tools to achieve the same task. 

In [1]:
sample_tool_call = [
    {
        "name": "w3",
        "arguments": {"email": "tchakra3@ibm.com"},
    },
    {
        "name": "author_workbench",
        "arguments": {"id": "$var1.id$"},
    },
    {
        "name": "hr_bot",
        "arguments": {"id": "$var1.id$", "email": "tchakra3@ibm.com"},
    },
    {
        "name": "concur",
        "arguments": {
            "travel_justification": "$var2.papers$",
        },
    },
]

Let's ask the refractor. The first couple of lines are initialization. Initialization of maps is optional and needs to happen once per set of tools -- if you do this the refractor will look to guess potential potential fixes that are not direct matches. 

In [ ]:
from altk.pre_tool.refraction.src.integration import Refractor

refractor_object = Refractor(tool_specs)
refractor_object.initialize_maps(mapping_threshold=0.65)

In [ ]:
result = refractor_object.refract(sample_tool_call, use_given_operators_only=True)



  var1 = w3(email="tchakra3@ibm.com")
  var2 = author_workbench(id="$var1.id$")
  var3 = hr_bot(id="$var1.id$", email="tchakra3@ibm.com")
- var4 = concur(travel_justification="$var2.papers$")
+ var4 = concur(employee_info="$var3.info$", travel_justification="$var2.papers$")
?               +++++++++++++++++++++++++++++



It's not only telling you that parameter is missing, but it's also giving you a potential fix that you have to add that parameter and you can possibly assign to that the output of one of the previous tool calls. 

The following signal will have to be True for a sequence of tool calls to be syntactically correct per their specs.

In [ ]:
assert result.report.determination is False

Under certain circumstances the fixed call maybe be executable as is, in which case case you just go ahead and do it. This is how you access the new tool call in structured form.

In [ ]:
result.corrected_function_call(
    memory={}, catalog=result.backing_data.catalog
).objectified

SequencingData(input='', output=[SequenceStep(name='w3', arguments={'email': 'tchakra3@ibm.com'}, label='var1', errors=[], response=None), SequenceStep(name='author_workbench', arguments={'id': '$var1.id$'}, label='var2', errors=[], response=None), SequenceStep(name='hr_bot', arguments={'id': '$var1.id$', 'email': 'tchakra3@ibm.com'}, label='var3', errors=[], response=None), SequenceStep(name='concur', arguments={'employee_info': '$var3.info$', 'travel_justification': '$var2.papers$'}, label='var4', errors=[], response=None)], var_result={}, errors=[], num_successful_calls=0)

Alternatively, you can also generate a much more detailed prompt stating what exactly was wrong and how it may be fixed. You can pass that on to your reflection stage if you have to but now with more detailed feedback on what went wrong.

In [ ]:
from altk.pre_tool.refraction.src.prompt_template import (
    generate_prompt,
    PromptType,
)

print(
    generate_prompt(
        result,
        sample_tool_call,
        result.backing_data.catalog,
        memory_objects={},
        prompt_type=PromptType.WITH_SUGGESTIONS,
    )
)

Please fix the provided tool call based on the issues outlined.

<tool_call>[
{'name': 'w3', 'arguments': {'email': 'tchakra3@ibm.com'}, 'label': 'var1'},
{'name': 'author_workbench', 'arguments': {'id': '$var1.id$'}, 'label': 'var2'},
{'name': 'hr_bot', 'arguments': {'id': '$var1.id$', 'email': 'tchakra3@ibm.com'}, 'label': 'var3'},
{'name': 'concur', 'arguments': {'travel_justification': '$var2.papers$'}, 'label': 'var4'}
]</tool_call>

The following are the identified issues:
Each issue is accompanied by guidance on how to fix it.
Consider the guidance, along with the provided tool specs, and memory, to come up with the final fixed tool call.

- Parameter employee_info is a required parameter for concur, but it is missing.
- Possible fix: Get value of parameter employee_info by calling hr_bot.
